# Redes neuronales (ejercicio)

### **Importante: comentar adecuadamente cada paso realizado**, relacionándolo con lo visto en la teoría.

## Parte 1: aplicación de redes neuronales a clasificación (análisis de sentimientos)

Se pide aplicar un modelo de redes neuronales al problema de decidir si una crítica de cine es positiva o negativa. Para ello volvemos a usar los datos de IMDB (Internet Movie Database) que vimos en el módulo 2 (modelo probabilístico).

Hacerlo usando los dos sistemas vistos, comparando los resultados:
* Scikit learn: usar `MLPClassifier`
* Keras con Tensorflow: usar `Sequential` y capas tipo `Dense` con la arquitectura adecuda.


Aunque ya hemos visto que los datos están disponibles en http://ai.stanford.edu/~amaas/data/sentiment/ , en este caso pedimos cargar los datos usando la utilidad `imdb`de Keras. Se puede consultar en el manual de Keras: https://keras.io/datasets/ Cargarlos con `imdb.load_data` y usar los datos cargados como punto de partida a este ejercicio (tanto para su aplicar scikit learn como para aplicar keras). Prestar atención al formato en el que se cargar, que no es el mismo que hamos visto hasta ahora.  

Los textos han de ser vectorizados para que se puedan ser procesados por una red. Para esto, tenemos varias alternativas, usar una de ellas:

* Vectorizando "manualmente", definiendo una función en python que lo haga.
* Vectorizadores de scikit learn (ya vistos)
* Herramientas de vectorización de keras: https://keras.io/preprocessing/text/

Mostrar algunas pruebas realizadas con distintas arquitecturas y/o hiperparámetros. No es necesario ser muy exhaustivo ni usar `GridSearchCV` en scikit learn ni el equivalente en keras. Tan solo mostrar alguna experimentación y ajuste manual.  

In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.neural_network import MLPClassifier, MLPRegressor
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten

# Scikit learn


Lo primero que tenemos que hacer como dice el enunciado es cargar el conjunto de datos y vectorizarlo, para esto he decidido usar la 3º opción (keras) y especificando un límite de palabras a 12000 ya que mi portatil no puede con tanto cálculo.

con keras he llamado a Tokenizer y con su método sequencestomatrix que me permite convetir la lista en matrices que contienen  la frecuencia de cada palabra, es decir, las matrices x_train y x_test_keras contienen una matriz donde me indica la frecuencia de cada palabra gracias al mode=count (con un mode=binary obtendriamos un valor de si esa palabra esta o no)

Después aplicamos redes neuronales usando MLPClassifier, le paso el número de capas internas, que en este primer caso serán dos de 16 cada una y límito el número de iteraciones a 1000, entrenamos el modelo y calculamos el rendimiento

Como dice el enunciado he probado a usar un número distinto de capas o la densidad de esta y en el tercer caso con una regularización distinta

In [2]:
max_words = 12000

imdb = keras.datasets.imdb.load_data(num_words=max_words)
(X_train, y_train), (X_test, y_test) = imdb

tokenizer = Tokenizer(num_words=max_words)
x_train_keras = tokenizer.sequences_to_matrix(X_train, mode='count')
x_test_keras = tokenizer.sequences_to_matrix(X_test, mode='count')

In [3]:
# Primero probamos con 2 capas internas de 16 y comprobamos el rendimiento tras entrenar el modelo
mlp_model = MLPClassifier(hidden_layer_sizes=(16, 16), max_iter=1000, random_state=42)
mlp_model.fit(x_train_keras, y_train)
score = mlp_model.score(x_test_keras, y_test)

print('Rendimiento con Scikit-learn: {}'.format(round(score, 3)))

Rendimiento con Scikit-learn: 0.86


In [4]:
# Eliminamos una capa y comprobamos si el rendimiento mejora
mlp_model = MLPClassifier(hidden_layer_sizes=(16), max_iter=1000, random_state=42)
mlp_model.fit(x_train_keras, y_train)
score = mlp_model.score(x_test_keras, y_test)

print('Rendimiento con Scikit-learn: {}'.format(round(score, 3)))

Rendimiento con Scikit-learn: 0.859


In [5]:
# Como no ha mejorado vamos a probar con una tercera capa y cambiamos la regularización a 0.1

mlp_model = MLPClassifier(hidden_layer_sizes=(32, 16 , 8),alpha=0.1, max_iter=1000, random_state=42)
mlp_model.fit(x_train_keras, y_train)
score = mlp_model.score(x_test_keras, y_test)

print('Rendimiento con Scikit-learn: {}'.format(round(score, 3)))

Rendimiento con Scikit-learn: 0.854


# Keras

Ahora vamos a hacer lo mismo pero usando Keras, hemos empleado capas tipo Dense para las intermedias y una función de activación relu.

Para la última capa al ser un problema de clasificación binaria usaremos como función de activación la sigmoide

In [6]:
keras_model = Sequential([
    Flatten(input_shape=(max_words,)),
    Dense(300, activation='relu'),
    Dense(100, activation='relu'),
    Dense(1, activation='sigmoid') 
])

In [7]:
# Evaluación del modelo de Keras y comprobación del rendimiento con un optimizador y un número de epochs específico
keras_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
keras_model.fit(x_train_keras, y_train, epochs=3, batch_size=512, validation_split=0.2)
keras_loss, keras_accuracy = keras_model.evaluate(x_test_keras, y_test)
print('Rendimiento con keras: {}'.format(round(keras_accuracy, 3)))


Epoch 1/3


40/40 [==============================] - 3s 46ms/step - loss: 0.6694 - accuracy: 0.6030 - val_loss: 0.6550 - val_accuracy: 0.6274
Epoch 2/3
40/40 [==============================] - 1s 34ms/step - loss: 0.6290 - accuracy: 0.6714 - val_loss: 0.6137 - val_accuracy: 0.6844
Epoch 3/3
782/782 [==============================] - 2s 3ms/step - loss: 0.5919 - accuracy: 0.7175
Rendimiento con keras: 0.718


In [8]:
# Como el rendimiento es malo, vamos a evaluar el modelo de Keras usando adam como el optimizador
keras_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
keras_model.fit(x_train_keras, y_train, epochs=3, batch_size=512, validation_split=0.2)
keras_loss, keras_accuracy = keras_model.evaluate(x_test_keras, y_test)
print('Rendimiento con keras: {}'.format(round(keras_accuracy, 3)))

Epoch 1/3
40/40 [==============================] - 5s 73ms/step - loss: 0.4301 - accuracy: 0.8140 - val_loss: 0.2947 - val_accuracy: 0.8902
Epoch 2/3
40/40 [==============================] - 2s 58ms/step - loss: 0.1814 - accuracy: 0.9340 - val_loss: 0.2827 - val_accuracy: 0.8960
Epoch 3/3
782/782 [==============================] - 2s 3ms/step - loss: 0.3846 - accuracy: 0.8731
Rendimiento con keras: 0.873


In [9]:
# En este tercer vamos a probar con adam ya que parece que da mejor resultado pero aumentando los epochs 
keras_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
keras_model.fit(x_train_keras, y_train, epochs=10, batch_size=512, validation_split=0.2)
keras_loss, keras_accuracy = keras_model.evaluate(x_test_keras, y_test)
print('Rendimiento con keras: {}'.format(round(keras_accuracy, 3)))

Epoch 1/10
40/40 [==============================] - 4s 69ms/step - loss: 0.0575 - accuracy: 0.9813 - val_loss: 0.4191 - val_accuracy: 0.8888
Epoch 2/10
40/40 [==============================] - 2s 57ms/step - loss: 0.0117 - accuracy: 0.9979 - val_loss: 0.4807 - val_accuracy: 0.8848
Epoch 3/10
40/40 [==============================] - 2s 53ms/step - loss: 0.0083 - accuracy: 0.9987 - val_loss: 0.5426 - val_accuracy: 0.8858
Epoch 4/10
40/40 [==============================] - 2s 54ms/step - loss: 0.0022 - accuracy: 0.9998 - val_loss: 0.6066 - val_accuracy: 0.8852
Epoch 5/10
40/40 [==============================] - 2s 52ms/step - loss: 7.4072e-04 - accuracy: 1.0000 - val_loss: 0.6570 - val_accuracy: 0.8846
Epoch 6/10
40/40 [==============================] - 2s 52ms/step - loss: 3.1861e-04 - accuracy: 1.0000 - val_loss: 0.6973 - val_accuracy: 0.8858
Epoch 7/10
40/40 [==============================] - 2s 54ms/step - loss: 1.7847e-04 - accuracy: 1.0000 - val_loss: 0.7354 - val_accuracy: 0.8872
E

## Parte 2: aplicación de redes neuronales a regresión (predicción del precio de la vivienda)

Se pide aplicar un modelo de redes neuronales al problema de predecir precios de vivienda usando el conjunto de datos  `Boston house prices`. 

Hacerlo usando los dos sistemas vistos, comparando los resultados:
* Scikit learn: usar `MLPRegressor`
* Keras con Tensorflow: usar nuevamente `Sequential` y capas tipo `Dense` con la arquitectura adecuada.

El conjunto de datos se puede cargar usando tanto scikit learn (`sklearn.datasets.load_boston`) como keras (`keras.datasets.boston_housing`). 

Como en la parte 1, se pide mostrar algunas pruebas de los resultados obtenidos usando distintas arquitecturas y/o hiperparámetros. 


Lo primero que vamos a hacer aqui es cargar y normalizar los valores del conjunto de boston, ya que como hemos visto en teoría son muy sensibles a las diferencias de magnitud de las unidades

Y crearemos el conjunto de entrenamiento (load_boston está ya desactualizado, debido a eso me sale un warning avisando).

Después simplemente probaremos variaciones para comprobar coomo fluctua el rendimiento para los distintos valores

In [10]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

# Cargarmos boston
boston = load_boston()
x, y = boston.data, boston.target

# Normalizar los datos para las redes
scaler = StandardScaler()
x = scaler.fit_transform(x)

# dividimos los datos
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

C:\Users\alarc\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i

In [11]:
# Modelo MLPRegressor de Scikit-learn con dos capas internas
mlp_regressor = MLPRegressor(hidden_layer_sizes=(50, 30), max_iter=500, random_state=42)
mlp_regressor.fit(x_train, y_train)
score = mlp_regressor.score(x_test, y_test)

print('Rendimiento con Scikit-learn MLPRegressor: {}'.format(round(score, 3)))

Rendimiento con Scikit-learn MLPRegressor: 0.729


In [12]:
# Modelo MLPRegressor de Scikit-learn con una única capa de 50 y aumentando el nº máxmo de iteraciones
mlp_regressor = MLPRegressor(hidden_layer_sizes=(50), max_iter=1000, random_state=42)
mlp_regressor.fit(x_train, y_train)
score = mlp_regressor.score(x_test, y_test)

print('Rendimiento con Scikit-learn MLPRegressor: {}'.format(round(score, 3)))

Rendimiento con Scikit-learn MLPRegressor: 0.807


In [13]:
# Modelo MLPRegressor de Scikit-learn añadiendo una tercera capa
mlp_regressor = MLPRegressor(hidden_layer_sizes=(50, 50, 20), max_iter=1000, random_state=42)
mlp_regressor.fit(x_train, y_train)
score = mlp_regressor.score(x_test, y_test)

print('Rendimiento con Scikit-learn MLPRegressor: {}'.format(round(score, 3)))

Rendimiento con Scikit-learn MLPRegressor: 0.824


# Keras

Por último vamos a aplicar keras sobre este conjunto, vamos a empezar con dos capas intermedias que luego variaremos para comprobar como se comporta el método y de salida una capa sin función de activación(lineal por defecto) ya que nos encontramos ante un problema de regresión

La función compile permite especificarle una serie de parámetros tales como un optimizador encargado de calcular los pesos óptimos o una función de pérdida ya que estamos en un problema de regresión
    

Finalmente entrenamos el modelo y evaluamos comprobando la pérdida en cada uno de los casos

In [14]:
# Modelo Sequential de Keras
keras_model_reg = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # No se utiliza función de activación(lineal por defecto) en la capa de salida para regresión
])

# Compilamos el modelo calculando el error cuadratico para ver el rendimiento
keras_model_reg.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

# Entrenamos el modelo de Keras
keras_model_reg.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
11/11 [==============================] - 1s 18ms/step - loss: 624.1054 - mean_squared_error: 624.1054 - val_loss: 542.5886 - val_mean_squared_error: 542.5886
Epoch 2/10
11/11 [==============================] - 0s 5ms/step - loss: 586.2067 - mean_squared_error: 586.2067 - val_loss: 509.5023 - val_mean_squared_error: 509.5023
Epoch 3/10
11/11 [==============================] - 0s 5ms/step - loss: 547.7231 - mean_squared_error: 547.7231 - val_loss: 471.7669 - val_mean_squared_error: 471.7669
Epoch 4/10
11/11 [==============================] - 0s 5ms/step - loss: 501.5439 - mean_squared_error: 501.5439 - val_loss: 425.5523 - val_mean_squared_error: 425.5523
Epoch 5/10
11/11 [==============================] - 0s 5ms/step - loss: 444.8545 - mean_squared_error: 444.8545 - val_loss: 369.1151 - val_mean_squared_error: 369.1151
Epoch 6/10
11/11 [==============================] - 0s 6ms/step - loss: 377.7365 - mean_squared_error: 377.7365 - val_loss: 304.5733 - val_mean_squared_error: 

In [15]:
# Calculamos la pérdida para el modelo de Keras
keras_score = keras_model_reg.evaluate(x_test, y_test)

4/4 [==============================] - 0s 1ms/step - loss: 74.6138 - mean_squared_error: 74.6138


In [16]:
# Vamos a probar ahora otro caso con una capa intermedia adicional 

keras_model_reg = Sequential([
    Dense(64, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)  # No se utiliza función de activación(lineal por defecto) en la capa de salida para regresión
])

# Compilamos el modelo calculando el error cuadratico para ver el rendimiento
keras_model_reg.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

# Entrenamos el modelo de Keras
keras_model_reg.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
11/11 [==============================] - 1s 17ms/step - loss: 632.9772 - mean_squared_error: 632.9772 - val_loss: 554.3345 - val_mean_squared_error: 554.3345
Epoch 2/10
11/11 [==============================] - 0s 5ms/step - loss: 605.3443 - mean_squared_error: 605.3443 - val_loss: 538.2869 - val_mean_squared_error: 538.2869
Epoch 3/10
11/11 [==============================] - 0s 5ms/step - loss: 585.3171 - mean_squared_error: 585.3171 - val_loss: 514.5273 - val_mean_squared_error: 514.5273
Epoch 4/10
11/11 [==============================] - 0s 5ms/step - loss: 553.1409 - mean_squared_error: 553.1409 - val_loss: 477.8574 - val_mean_squared_error: 477.8574
Epoch 5/10
11/11 [==============================] - 0s 4ms/step - loss: 505.9354 - mean_squared_error: 505.9354 - val_loss: 425.5505 - val_mean_squared_error: 425.5505
Epoch 6/10
11/11 [==============================] - 0s 6ms/step - loss: 439.4791 - mean_squared_error: 439.4791 - val_loss: 354.0451 - val_mean_squared_error: 

In [17]:
# Evaluar el rendimiento en el modelo de Keras y podemos comprobar que en este segundo caso la pérdida es menor
keras_score = keras_model_reg.evaluate(x_test, y_test)

4/4 [==============================] - 0s 0s/step - loss: 58.2393 - mean_squared_error: 58.2393
